In [ ]:
import os
currentdir = os.getcwd()
correctdir = currentdir.rsplit('tv2ds/',1)[0]
os.chdir(correctdir)

from tv2ds.ds_lib import notebook_prodrun
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import tvsquared.settings
from tvsquared.lib.request import Request
from tvsquared.lib.athena import AthenaDatabase
pd.options.display.float_format = '{:.2f}'.format
pd.set_option('display.max_colwidth', None)

In [ ]:
notebook_prodrun.set_env('US', prodrunenv='PROD')
crosswalk_suffix = os.environ.get('TV2PRODRUNENV').lower()
crosswalk_suffix

'produsa'

In [ ]:
def make_request(clientid, brandid = 1, datefrom=False, dateto=False):
    if datefrom and dateto:
        request = Request(clientarg=False, brandarg=False, datesarg=False)
        datefrom = datetime.datetime.strptime(datefrom, '%Y-%m-%d')
        dateto = datetime.datetime.strptime(dateto, '%Y-%m-%d')
        request = request.init(partnerid=None, clientid=clientid, brandid=brandid,datefrom=datefrom, dateto=dateto,loglevel=-1, extargs=None, request=None, usespark=None, readPreference=None,prodrun=False)
        return(request)
    else:
        request = Request(clientarg=False, brandarg=False, datesarg=False)
        request = request.init(partnerid=None, clientid=clientid, brandid=brandid,datefrom=None, dateto=None,loglevel=-1, extargs=None, request=None, usespark=None, readPreference=None,prodrun=False)
        return(request)


def query_athena(request, query, copy_to_local=False):
    athena_db = AthenaDatabase.get_client_database(request)
    query_results = AthenaDatabase.execute_query(athena_db, query, copy_to_local=copy_to_local)
    df = pd.DataFrame(query_results)
    return(df)

In [ ]:
client = {'clientid': 9306}
request = make_request(client['clientid'])

In [ ]:
query_exposed_march = '''with filtered_hh as (
   select key_value as mapped_tv2_hhid
   from {crosswalk_suffix}_modeldata.crosswalk
   where yy='{yy}' and mm='{mm}' and dd='{dd}' and key_name ='tv2_hhid' and vendor_name = '{vendor_name}' and excluded_stamp is null
),
campaign_universe as (
   select cw.mapped_tv2_hhid
   from {crosswalk_suffix}_modeldata.pre_crosswalk cw
   join filtered_hh fh on fh.mapped_tv2_hhid = cw.mapped_tv2_hhid
   where
       first_seen <= timestamp '{dateto}' + interval '1' day
       and last_seen >= timestamp '{datefrom}'
       and yy='{yy}' and mm='{mm}' and dd='{dd}' and key_name ='ip_address' and vendor_name = '{vendor_name}'
),
client_eventlog as (
   select *
   from {crosswalk_suffix}_client_data.eventlog
   where client = '{clientid}'
),
hh_impressed_in_campaign as (
   select
       distinct crosswalk_link_id as mapped_tv2_hhid
   from campaign_universe ex
   join client_eventlog ev on ev.crosswalk_link_id = ex.mapped_tv2_hhid
   where
       event_class='impression'
       and datadatetime between timestamp '{datefrom}' and timestamp '{dateto}' + interval '1' day
       and in_scope
),
n_hh_impressed_visited as (
   select distinct 
      mapped_tv2_hhid
   from client_eventlog
   join hh_impressed_in_campaign on crosswalk_link_id = mapped_tv2_hhid
   where
       event_class ='response' and event= 'all response'
       and datadatetime between timestamp '{datefrom}' and timestamp '{dateto}' + interval '6' day + interval '1' day
       and in_scope
),
hh_impressed_visited as (
   select
      impressed.mapped_tv2_hhid 
      ,(case when visited.mapped_tv2_hhid is not null then 1 else 0 end) as visited 
   from hh_impressed_in_campaign impressed
   left join n_hh_impressed_visited visited on impressed.mapped_tv2_hhid = visited.mapped_tv2_hhid
),
n_visited_impressed as (
   select SUM(visited) as exp_visited
   from hh_impressed_visited
),
n_hh_impressed as (
   select CAST(count(distinct mapped_tv2_hhid) AS double) as exp_hh
   from hh_impressed_visited
),
final_results as (
   select *
   from n_hh_impressed
   cross join n_visited_impressed
)
select
   *
from hh_impressed_visited

'''.format(vendor_name='inscape', yy='2022', mm='03',dd='28',crosswalk_suffix='produsa',dateto='2022-03-31', datefrom='2022-03-01', clientid='9306') 

In [ ]:
df_exposed_march= query_athena(request, query_exposed_march) #--> 

The history saving thread hit an unexpected error (OperationalError('database is locked')).History will not be written to the database.


In [ ]:
df_exposed_march.shape 

(1137786, 2)

In [ ]:
df_exposed_march.head()